In [1]:
# ─────────────────────────────────────────────────────────────────────────────
# Additional Questions — Department Gauges (no averaging)
# ─────────────────────────────────────────────────────────────────────────────
import pathlib, re, unicodedata
import numpy as np, pandas as pd, plotly.graph_objects as go

# 1) Paths (match your project)
ROOT_DIR  = pathlib.Path(r"C:\Repositories\odi-data-visualization")
DATA_FILE = ROOT_DIR / "data" / "additional_questions.csv"
PLOTS_DIR = ROOT_DIR / "plots"    # same folder tree as existing gauges
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

# 2) Helpers (same as your gauge script)
SAFE  = re.compile(r"[^0-9A-Za-z _()\-.’']+")
def normalize_quotes(s: str) -> str:
    s = unicodedata.normalize("NFKC", str(s))
    return s.replace("’", "'").strip()

def clean(s: str, n=80) -> str:
    s = normalize_quotes(s)
    s = SAFE.sub("_", s)           # strip bad chars
    s = re.sub(r"\s+", " ", s)     # collapse spaces
    return s[:n].strip()

# 3) Color bands (1–5)
BANDS = [
    (1.00, 3.29, "#e53935"),   # red
    (3.30, 3.49, "#fb8c00"),   # orange
    (3.50, 3.79, "#fdd835"),   # yellow
    (3.80, 5.00, "#43a047"),   # green
]
def band_color(v, default="#999999"):
    try:
        v = float(v)
    except Exception:
        return default
    for lo, hi, col in BANDS:
        if lo <= v <= hi:
            return col
    return default

# 4) Load table (read strings first to keep counts row as text)
df_raw = pd.read_csv(DATA_FILE, dtype=str)
df_raw.columns = [normalize_quotes(c) for c in df_raw.columns]

metric_names = df_raw.iloc[:, 0].astype(str)   # first col = question text (includes header)
departments  = df_raw.columns[1:]              # rest are departments

# Convert department columns to numeric values; counts → NaN
df = df_raw.copy()
for col in departments:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# 5) Group mapping (same logic; forgiving text match)
ELECTED_PATTERNS = (
    r"\bcity\s*auditor\b",
    r"\bthe\s*city\s*auditor\b",
    r"\bcity\s*attorney\b",
    r"\bcity\s*council\b",
)
def group_of(dept: str) -> str:
    norm = normalize_quotes(dept).casefold()
    if any(re.search(p, norm) for p in ELECTED_PATTERNS):
        return "elected_officials"
    return "mayors_office"

# 6) Gauge builder (ticks 1..5; 0.5 minor labels blank; white background)
def build_gauge(score, title):
    vals = np.arange(1, 5.01, 0.5)
    txt  = [f"{v:.0f}" if float(v).is_integer() else "" for v in vals]
    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=float(score),
        title={"text": title, "font": {"size": 18, "color": "#1f3d5a"}},
        number={"font": {"size": 38, "color": "#1f3d5a"}},
        gauge={
            "axis": {"range": [1, 5],
                     "tickmode": "array",
                     "tickvals": vals,
                     "ticktext": txt,
                     "ticks": "outside",
                     "ticklen": 8,
                     "tickwidth": 2,
                     "tickcolor": "#444",
                     "tickfont": {"size": 12}},
            "bar": {"color": band_color(score), "thickness": 0.45},
            "bgcolor": "#FFFFFF",
            "borderwidth": 1,
            "bordercolor": "#444"
        }
    ))
    fig.update_layout(
        height=450, width=450,
        plot_bgcolor="#FFFFFF", paper_bgcolor="#FFFFFF",
        margin=dict(t=60, b=20, l=20, r=20)
    )
    return fig

def save_png(fig, path: pathlib.Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(path, scale=2)

# 7) Build gauges for EVERY department/question (skip counts row)
print("Generating Additional Questions gauges…")
for dept in departments:
    grp = group_of(dept)
    dept_dir = PLOTS_DIR / grp / clean(dept)

    # Series of scores indexed by question text
    s = pd.Series(df[dept].values, index=metric_names)

    for metric, score in s.items():
        # Skip the counts row "(n = …)" and any NaNs
        if pd.isna(score) or metric.strip().lower() in {"outcome"}:
            continue

        title = f"{dept} — {metric}"
        fig = build_gauge(score, title)

        # Prefix "AQ_" to make these easy to tell apart from the core subscales
        fname = f"AQ_{clean(metric)}.png"
        save_png(fig, dept_dir / fname)
        print(f"  • {grp} / {dept} — {metric} → {fname}")

print("\nDone. Files saved under:", PLOTS_DIR)

Generating Additional Questions gauges…
  • mayors_office / Building and Zoning Services — I believe employees are treated fairly, regardless of differences (e.g., race, gender, ability). → AQ_I believe employees are treated fairly_ regardless of differences (e.g._ race_ g.png
  • mayors_office / Building and Zoning Services — I feel like I belong at City of Columbus. → AQ_I feel like I belong at City of Columbus..png
  • mayors_office / Building and Zoning Services — I feel connected as an employee to City of Columbus. → AQ_I feel connected as an employee to City of Columbus..png
  • mayors_office / Building and Zoning Services — I feel comfortable with being myself when I am at work. → AQ_I feel comfortable with being myself when I am at work..png
  • mayors_office / Building and Zoning Services — I can be successful as my authentic self at City of Columbus. → AQ_I can be successful as my authentic self at City of Columbus..png
  • mayors_office / Building and Zoning Services — I fee